<center><h1>Group 2404 </h1></center>


<center><style>
table {
    font-size: 24px;
}
</style></center>

| Last Name          | First Name            |Student Number|
|--------------------|-----------------------|----------------|
| Malagoli           | Pietro                |2125711         |
| Boccanera          | Eugenia               |2109310         |
| Braidi             | Federico              |2122169         |
| Lovato             | Matteo                |2104269         |           

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
# Gradient Boosting 
from sklearn.ensemble import GradientBoostingClassifier
# XGBoost 
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance, to_graphviz, plot_tree
print("XGBoost version:",xgboost.__version__)
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split, cross_val_score

mycmap = "winter"
mpl.rcParams['image.cmap'] = mycmap
plt.rcParams['font.size'] = 13

In [ ]:
np.random.seed(12345)

dname="./DATA/"
str0="_XGB_24.dat"
fnamex=dname+'x'+str0
fnamey=dname+'y'+str0
x = np.loadtxt(fnamex, delimiter=" ",dtype=float)
y = np.loadtxt(fnamey)
y = y.astype(int)
N,L = len(x), len(x[0])

N_train = int(0.75*N)
x_train,y_train = x[:N_train],y[:N_train]
x_test,y_test = x[N_train:],y[N_train:]
print(f"N={N}, N_train={N_train}, L={L}")

## Point 3

In [ ]:
def scat(ax,x,y,i=0,j=1,s=4,title=""):
    ax.scatter(x[:,i],x[:,j],s=s,c=y)
    ax.set_xlabel(f"feature {i}")
    ax.set_ylabel(f"feature {j}")
    ax.set_title(title)

for n in range(5):
    print(x[n],y[n])

fig,AX = plt.subplots(2,2,figsize=(8.5,8.1))
scat(AX[0,0],x_train,y_train,title="Train")
scat(AX[0,1],x_train,y_train,i=2,j=3,title="Train")
scat(AX[1,0],x_test,y_test,title="Test")
scat(AX[1,1],x_test,y_test,i=2,j=3,title="Test")
fig.tight_layout()
plt.show()

In [ ]:
def classify(clf=GradientBoostingClassifier(),show=False):
    # GradientBoostingClassifier():
    #   n_estimators = 100 (default)
    #   loss function = deviance(default) used in Logistic Regression
    # XGBClassifier()
    #   n_estimators = 100 (default)
    #   max_depth = 3 (default?)
    clf.fit(x_train,y_train)
    y_hat = clf.predict(x_test)
    
    #print("errors: {:.2f}%   Accuracy={:.3f}".format(100*(1-clf.score(x_test, y_test)),clf.score(x_test, y_test)))
    
    if show: 
        S=50
        dx = 1
        x_seq=np.arange(-S,S+dx,dx)
        nx = len(x_seq)
        x_plot=np.zeros((nx*nx,L))
        q=0
        for i in range(nx):
            for j in range(nx):
                x_plot[q,:2] = [x_seq[i],x_seq[j]]
                q+=1
        y_plot= clf.predict(x_plot)

        fig,AX = plt.subplots(1,2,figsize=(8.2,4))
        scat(AX[0],x_plot[:],y_plot,s=dx,title="predicted")
        scat(AX[1],x_train[:],y_train,title="training set")
        fig.tight_layout()
        plt.show()     
        dump_list = clf.get_booster().get_dump()
        num_trees = len(dump_list)
        print("num_trees=",num_trees)
        
        fig, AX = plt.subplots(2,1,figsize=(12, 5))
        for i in range(min(2,num_trees)):
            ax=AX[i]
            plot_tree(clf, num_trees=i, ax=ax)
        fig.savefig("DATA/tree-classif.png", dpi=400, pad_inches=0.02)   
        plt.show()
    
    return clf.score(x_test,y_test)

In [ ]:
def build_nn(num_layers,num_neurons,activation_function):
    input_shape = x.shape[1]
    model_nn = Sequential()
    model_nn.add(Dense(units=num_neurons, activation=activation_function, input_shape=(input_shape,)))
    for i in range(num_layers-1):
        model_nn.add(Dense(units=num_neurons, activation=activation_function))
    model_nn.add(Dense(units=2,activation='softmax'))
    opt = keras.optimizers.Adam()
    model_nn.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    return model_nn

num_layers=[2]                      #num of nn layers(input and output excluded)
num_neurons=[20]                        #num of neurons per layer
activation_functions=['relu']         #activation function for the hidden layers
max_acc=0
max_desc=""
perc=0.7
n_rep=30


for nl in num_layers:
    for nn in num_neurons:
        for act in activation_functions:
            accuracies_nn=[]
            for i in range(n_rep):
                num_indices=int(perc*len(x))        #get quantity of data points in training
                data_indices = np.arange(len(x))    #get indices
                np.random.shuffle(data_indices)     #shuffle indices
                train_indices = data_indices[:num_indices]  #get training set indices
                test_indices = data_indices[num_indices:]   #get test set indices
                
                x_train, y_train = x[train_indices], y[train_indices]
                x_test, y_test = x[test_indices], y[test_indices]
                y_train = keras.utils.to_categorical(y_train, num_classes=2)
                y_test = keras.utils.to_categorical(y_test, num_classes=2)
                print(len(x_train))
                print(len(x_test))
                
                print(f"--RUNNING ({i+1}/{n_rep})--n_lay: {nl}, n_neu: {nn}, act: {act}")
                model=build_nn(num_layers=nl,num_neurons=nn,activation_function=act)
                model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=0)
                accuracy_nn = model.evaluate(x_test, y_test, verbose=0)[1]  #accuracy is at index 1
                accuracies_nn.append(accuracy_nn)
                print(accuracy_nn)
            mean_acc=np.mean(accuracies_nn)
            if(mean_acc>max_acc):
                max_acc = mean_acc
                max_desc = f"n_lay: {nl}, n_neu: {nn}, act: {act}"
            print(model.summary())
print(f"Best mean accuracy ({max_acc}) obtained by the model with parameters: {max_desc}")


In [ ]:
def build_best_xg():
    model_xg = XGBClassifier(seed=1,
        objective='binary:logistic', 
        importance_type="gain", #weight, cover, ...
        reg_lambda=0.2,
        gamma=0.1, 
        n_estimators=30,
        max_depth=10,
        learning_rate=0.1)
    return model_xg

def build_best_nn():
    input_shape = x.shape[1]
    model_nn = Sequential()
    model_nn.add(Dense(20, activation='relu', input_shape=(input_shape,)))
    model_nn.add(Dense(20, activation='relu'))
    model_nn.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam()
    model_nn.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    return model_nn

In [ ]:
train_perc = [0.03,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]  #percentage of data to be used in training 
n_rep = 30                          #number of times training-test is repeated to get the statistic
mean_acc_best_xg = []
mean_acc_best_nn = []

for perc in train_perc:
    accuracies_best_xg = []
    accuracies_best_nn = []
    model_best_nn=build_best_nn()
    model_best_xg=build_best_xg()
    print(model_best_nn.summary())
    for i in range(n_rep):
        num_indices=int(perc*len(x))        #get quantity of data points in training
        data_indices = np.arange(len(x))    #get indices
        np.random.shuffle(data_indices)     #shuffle indices
        train_indices = data_indices[:num_indices]  #get training set indices
        test_indices = data_indices[num_indices:]   #get test set indices
        #print(len(train_indices))
        #print(len(test_indices))

        x_train, y_train = x[train_indices], y[train_indices]
        x_test, y_test = x[test_indices], y[test_indices]

        #XGBoost
        model_best_xg.fit(x_train, y_train)
        accuracy_best_xg = model_best_xg.score(x_test, y_test)
        accuracies_best_xg.append(accuracy_best_xg)

        #Neural Network
        y_train = keras.utils.to_categorical(y_train, num_classes=2)
        y_test = keras.utils.to_categorical(y_test, num_classes=2)
        model_best_nn.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=0)
        prova=model_best_nn.evaluate(x_test, y_test, verbose=0)
        print(prova)
        accuracy_best_nn = prova[1]  #accuracy is at index 1
        accuracies_best_nn.append(accuracy_best_nn)

    mean_acc_best_xg.append(np.mean(accuracies_best_xg))
    mean_acc_best_nn.append(np.mean(accuracies_best_nn))
    print(f"Average accuracy on test set with XGBoost, cross-validation and {perc} of the data as training: {np.mean(accuracies_best_xg)}")
    print(f"Average accuracy on test set with dense NN and cross-validation and {perc} of the data as training: {np.mean(accuracies_best_nn)}")


In [ ]:
print(mean_acc_best_xg)
print(mean_acc_best_nn)

plt.plot([ perc*100 for perc in train_perc],mean_acc_best_xg,c="r",label="XGBoost")
plt.plot([ perc*100 for perc in train_perc],mean_acc_best_nn,c="g",label="FFNeuralNetwork")

plt.xlabel("Percentage of dataset used for training")
plt.ylabel("Average validation accuracy")
plt.legend()
plt.show()